# Projeto 2 - Sistema de Recomendação para Aplicativos de Contabilidade

Neste projeto o objetivo é construir um sistema de recomendação com a arquitetura Graph Attention Network – GAT através do mecanismo de atenção em uma rede convolucional de grafos. O projeto é em Linguagem Python com PyTorch e PyTorch Geometric.Ao final do projeto o sistema será capaz de recomendar aplicativos de Contabilidade.

## 1. Carga e Instalação dos Pacotes

In [2]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da linguagem Python usada neste Jupyter Notebook é: ', python_version())

Versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [3]:
# Instalação Pytorch
!pip install -q torch==2.0.0

In [4]:
# Instalação Pytorch Geometric
!pip install -q torch_geometric==2.3.1

In [5]:
# Carga dos Pacotes
# Manipulação de Dados
import pandas as pd
import numpy as np

# Pytorch
import torch
import torch.nn.functional as F

# Pytorch-Geometric
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.utils import to_undirected
from torch_geometric.loader import DataLoader

#Scikit-Learn
import sklearn
from sklearn.model_selection import train_test_split

# Remoção de Warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 2 - Sistema de Recomendação App Contabilidade" --iversions

Author: Projeto 2 - Sistema de Recomendação App Contabilidade

sklearn        : 1.0.2
torch_geometric: 2.3.1
torch          : 2.0.0
numpy          : 1.21.5
pandas         : 1.4.4



## 2. Preparação do Conjunto de Dados

In [8]:
# Dataframe de usuários
users = pd.DataFrame({"user_id": [0, 1, 2, 3]})

# Dataframe de apps
aplicativos = pd.DataFrame({"app_id": [0, 1, 2]})

# Dataframe de avaliações (ratings)
ratings = pd.DataFrame({"user_id": [0, 1, 1, 2, 3],
                        "app_id": [0, 0, 1, 2, 2],
                        "rating": [4, 5, 3, 2, 4]})

In [9]:
# Visualiza dados de usuários
users

,user_id
0,0
1,1
2,2
3,3


In [10]:
# Visualiza dados de aplicativos
aplicativos

,app_id
0,0
1,1
2,2


In [11]:
# Visualia dados de ratings
ratings

,user_id,app_id,rating
0,0,0,4
1,1,0,5
2,1,1,3
3,2,2,2
4,3,2,4


In [12]:
# Converta os IDs dos aplicativos para evitar confusão com os IDs dos usuários
ratings["app_id"] += users.shape[0] - 1

In [13]:
# Visualiza as alterações em ratings
ratings

,user_id,app_id,rating
0,0,3,4
1,1,3,5
2,1,4,3
3,2,5,2
4,3,5,4


## 3. Pré-Processamento dos Dados em Formato de Grafo

In [14]:
# Divide os dados em conjuntos de treinamento e teste
train_ratings, test_ratings = train_test_split(ratings, test_size = 0.2, random_state = 42)

In [15]:
# Prepara os nodes de treino
train_source_nodes = torch.tensor(train_ratings['user_id'].values, dtype = torch.long) #torch.long é para aumentar a precisão do cálculo, igual o float32
train_target_nodes = torch.tensor(train_ratings['app_id'].values, dtype = torch.long)

In [16]:
# Visualiza a fonte dos nodes
train_source_nodes

tensor([3, 1, 0, 2])

In [17]:
# Visualiza o target dos nodes
train_target_nodes

tensor([5, 4, 3, 5])

Os dados separados em treino e teste são tranformados nos nodes, ou nós, para o processamente com os grafos. No exemplo acima, temos os nós dos usuários e os nós dos app, porém ainda não temos nada com o que ligar as duas partes.

In [24]:
# Prepara os edges (arestas) de treino
train_edge_index = torch.stack([train_source_nodes, train_target_nodes], dim = 0)
train_edge_index = to_undirected(train_edge_index)
#train_edge_attr = torch.tensor(train_ratings['rating'].values, dtype = torch.float32).view(-1,1)

In [25]:
# Visualiza o index
train_edge_index

tensor([[0, 1, 2, 3, 3, 4, 5, 5],
        [3, 4, 5, 0, 5, 1, 2, 3]])

In [21]:
# Visualiza a aresta
train_edge_attr

tensor([[4.],
        [3.],
        [4.],
        [2.]])

Esta etapa criou as arestas, ou seja, o que liga os nodes entre si. Aqui temos o caso a aresta bidirecional, que siginifica que o modelo irá aprender da fonte para o target e ao contrário também. Por isso no index, temos mais registros que nos nodes de treino, os números são os mesmos dos nodes de treino. Então temos o sentido indo e voltando.

Nesse caso, a avaliação determina qual será a distância da aresta para ligar os nodes. 

In [26]:
# Repete o processo anterior para os dados de teste
test_source_nodes = torch.tensor(test_ratings["user_id"].values, dtype = torch.long)
test_target_nodes = torch.tensor(test_ratings["app_id"].values, dtype = torch.long)
test_edge_index = torch.stack([test_source_nodes, test_target_nodes], dim = 0)
test_edge_index = to_undirected(test_edge_index)
test_edge_attr = torch.tensor(test_ratings["rating"].values, dtype = torch.float32).view(-1, 1)

In [27]:
# Número de nodes
num_nodes = users.shape[0] + aplicativos.shape[0]

# Visualiza o número de nodes
num_nodes

7

In [28]:
# Cria os atributos de aresta do conjunto de treinamento
# Função Data do Torch-Geometric
# Função cria o conjunto de dados com grafo, cria uma matriz
train_data = Data(x = torch.eye(num_nodes, dtype = torch.float32),
                  edge_index = train_edge_index, #Indices das arestas
                  edge_attr = train_edge_attr, # Atributos das arestas
                  y = train_edge_attr) # Avaliação dos usuários, o que o modelo vai prever

In [29]:
# Visualiza o grafo
train_data

Data(x=[7, 7], edge_index=[2, 8], edge_attr=[4, 1], y=[4, 1])

##### Descrição dos Itens no Grafo

Essa representação acima é típica de um objeto Data do PyTorch Geometric, que armazena as informações de um grafo em um formato específico. Vamos entender cada componente dessa representação:

x=[7, 7]: Essa é a matriz de atributos dos nós. O primeiro valor (7) indica o número de nós no grafo, e o segundo valor (7) representa o número de atributos (características) para cada nó. Portanto, a matriz de atributos dos nós terá dimensões 7x7.

edge_index=[2, 8]: Essa é a matriz de índices de arestas, que indica as conexões entre os nós. A matriz tem duas linhas (valor 2), onde cada coluna representa uma aresta. O número 8 indica que existem 8 arestas no grafo.

edge_attr=[4, 1]: Essa é a matriz de atributos das arestas. O primeiro valor (4) indica o número de arestas no grafo, e o segundo valor (1) representa o número de atributos (características) para cada aresta. 

y=[4, 1]: Essa é a matriz de rótulos (labels) dos nós ou arestas, geralmente usada como a variável alvo (target) em problemas de aprendizado supervisionado. O primeiro valor (4) indica o número de rótulos, e o segundo valor (1) indica que cada rótulo é unidimensional (escalar). 

In [30]:
# Cria os atributos de aresta do conjunto de teste
test_data = Data(x = torch.eye(num_nodes, dtype = torch.float32), 
                 edge_index = test_edge_index, 
                 edge_attr = test_edge_attr, 
                 y = test_edge_attr)

In [31]:
# Cria os dataloaders (o que é requerido pelo PyTorch)
train_data_loader = DataLoader([train_data], batch_size = 1)
test_data_loader = DataLoader([test_data], batch_size = 1)

## 4. Construção do Modelo